In [1]:
from lang import load_data_int_seq
from models.rl.env import IntegerSequenceEnv
from models.rl.agents.agent_mcts.main import train_mcts


%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 999.0

In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()
train = train[:1]
env = IntegerSequenceEnv({"data": train, "output_length": 9, "input_lang": input_lang, "output_lang": output_lang, "penalty_at_end": True})

self.state
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8]


In [31]:
train_mcts(env, num_epochs=500, policy=None, test=False)

Testing Agent

Training Episodes: 0
Action: 2
State:
[[-1 -1 -1 -1 -1 -1 -1 -1 -1  0  1  2  3  4  5  6  7  8]
 [ 9 -1 -1 -1 -1 -1 -1 -1 -1  0  1  2  3  4  5  6  7  8]
 [ 9  4 -1 -1 -1 -1 -1 -1 -1  0  1  2  3  4  5  6  7  8]
 [ 9  4  5 -1 -1 -1 -1 -1 -1  0  1  2  3  4  5  6  7  8]
 [ 9  4  5  4 -1 -1 -1 -1 -1  0  1  2  3  4  5  6  7  8]
 [ 9  4  5  4  2 -1 -1 -1 -1  0  1  2  3  4  5  6  7  8]
 [ 9  4  5  4  2  4 -1 -1 -1  0  1  2  3  4  5  6  7  8]
 [ 9  4  5  4  2  4  2 -1 -1  0  1  2  3  4  5  6  7  8]
 [ 9  4  5  4  2  4  2  1 -1  0  1  2  3  4  5  6  7  8]
 [ 9  4  5  4  2  4  2  1  2  0  1  2  3  4  5  6  7  8]]
['9', '-', '1', '-', '3', '-', '3', '+', '3']
Step: 9
Return: -100.0


# HillClimbingEnv

In [ ]:
from models.rl.agents.agent_mcts.env_archive.hill_climbing_env import HillClimbingEnv

In [ ]:
env = HillClimbingEnv()

In [ ]:
train_mcts(env, num_epochs=2, policy=policy)